In [1]:
import tensorflow as tf
import autokeras as ak
import numpy as np
import time
import sys  
sys.path.insert(0, '../')

from soundings.preprocessing import dataloader as dl
from soundings.plotting import radiosonde_plotting, results
from soundings.deep_learning import mlutilities as ml

PROCESSED_VOL = '/mnt/data1/stock/mlsoundings/processed/'

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

%load_ext autoreload
%autoreload 2

In [4]:
gpus = tf.config.get_visible_devices('GPU')
for device in gpus:
    print(device)
    # tf.config.experimental.set_visible_devices(device, 'GPU')
    tf.config.experimental.set_memory_growth(device, True)
print()

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')



In [2]:
raob, rap, goes, rtma, sonde_files = dl.load_preprocessed_samples(PROCESSED_VOL, shuffle=True)

raob.shape, rap.shape, goes.shape, rtma.shape

total of 735 samples!


FloatProgress(value=0.0, max=735.0)

time: 20.021, avg: 0.027 seconds


((735, 256, 4), (735, 256, 4), (735, 3, 3, 8), (735, 3, 3, 3))

In [22]:
Xtrain, Ttrain, Xtest, Ttest = ml.partition(rap, raob, (0.8,0.2), shuffle=False)

input_dims = [dl.PRESSURE, dl.TEMPERATURE]

Xtrain = np.expand_dims(Xtrain[:, :, input_dims], axis=-2)
Ttrain = Ttrain[:, :, dl.TEMPERATURE]
Xtest = np.expand_dims(Xtest[:, :, input_dims], axis=-2)
Ttest = Ttest[:, :, dl.TEMPERATURE]

Xtrain.shape, Ttrain.shape, Xtest.shape, Ttest.shape

((588, 256, 1, 2), (588, 256), (147, 256, 1, 2), (147, 256))

In [23]:
clf = ak.ImageRegressor(
    max_trials=20,
    metrics = [tf.keras.metrics.RootMeanSquaredError()]
)
# Note: this is still slower...b
# distribution_strategy=tf.distribute.MirroredStrategy()

start_t = time.time()
clf.fit(
    Xtrain,
    Ttrain,
    validation_split=0.15,
    epochs=200,
)
print(f'Finished in {time.time() - start_t:.3f} seconds.')

# Predict with the best model.
# Y = clf.predict(Xtest)

# Evaluate the best model with testing data.
print(clf.evaluate(Xtest, Ttest))

Trial 20 Complete [00h 01m 16s]
val_loss: 29.85206413269043

Best val_loss So Far: 2.9359214305877686
Total elapsed time: 00h 52m 08s
INFO:tensorflow:Oracle triggered exit
Epoch 1/200
19/19 [==============================] - 2s 107ms/step - loss: 263.9059 - root_mean_squared_error: 15.7450
Epoch 2/200
19/19 [==============================] - 1s 76ms/step - loss: 26.7016 - root_mean_squared_error: 5.1674
Epoch 3/200
19/19 [==============================] - 1s 76ms/step - loss: 26.3803 - root_mean_squared_error: 5.1362
Epoch 4/200
19/19 [==============================] - 1s 76ms/step - loss: 25.7643 - root_mean_squared_error: 5.0759
Epoch 5/200
19/19 [==============================] - 1s 76ms/step - loss: 23.8490 - root_mean_squared_error: 4.8835
Epoch 6/200
19/19 [==============================] - 1s 76ms/step - loss: 20.9277 - root_mean_squared_error: 4.5747
Epoch 7/200
19/19 [==============================] - 1s 76ms/step - loss: 18.0982 - root_mean_squared_error: 4.2542
Epoch 8/200
1

19/19 [==============================] - 1s 77ms/step - loss: 3.1717 - root_mean_squared_error: 1.7809
Epoch 71/200
19/19 [==============================] - 1s 77ms/step - loss: 3.1767 - root_mean_squared_error: 1.7823
Epoch 72/200
19/19 [==============================] - 1s 77ms/step - loss: 3.1219 - root_mean_squared_error: 1.7669
Epoch 73/200
19/19 [==============================] - 1s 77ms/step - loss: 3.1312 - root_mean_squared_error: 1.7695
Epoch 74/200
19/19 [==============================] - 1s 77ms/step - loss: 3.1301 - root_mean_squared_error: 1.7692
Epoch 75/200
19/19 [==============================] - 1s 77ms/step - loss: 3.1045 - root_mean_squared_error: 1.7620
Epoch 76/200
19/19 [==============================] - 1s 77ms/step - loss: 3.0541 - root_mean_squared_error: 1.7476
Epoch 77/200
19/19 [==============================] - 1s 77ms/step - loss: 3.0396 - root_mean_squared_error: 1.7435
Epoch 78/200
19/19 [==============================] - 1s 77ms/step - loss: 3.0613 - r

19/19 [==============================] - 1s 76ms/step - loss: 2.1033 - root_mean_squared_error: 1.4503
Epoch 141/200
19/19 [==============================] - 1s 77ms/step - loss: 2.1134 - root_mean_squared_error: 1.4537
Epoch 142/200
19/19 [==============================] - 1s 77ms/step - loss: 2.0988 - root_mean_squared_error: 1.4487
Epoch 143/200
19/19 [==============================] - 1s 77ms/step - loss: 2.0841 - root_mean_squared_error: 1.4436
Epoch 144/200
19/19 [==============================] - 1s 77ms/step - loss: 2.0582 - root_mean_squared_error: 1.4347
Epoch 145/200
19/19 [==============================] - 1s 77ms/step - loss: 2.0378 - root_mean_squared_error: 1.4275
Epoch 146/200
19/19 [==============================] - 1s 77ms/step - loss: 2.1945 - root_mean_squared_error: 1.4814
Epoch 147/200
19/19 [==============================] - 1s 77ms/step - loss: 2.0642 - root_mean_squared_error: 1.4367
Epoch 148/200
19/19 [==============================] - 1s 77ms/step - loss: 2.

In [26]:
Y = clf.predict(Xtest)
ml.rmse(Ttest, Y)

2.340935654112626

In [28]:
clf.export_model().summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 1, 2)]       0         
_________________________________________________________________
resizing (Resizing)          (None, 256, 32, 2)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 32, 3)        9         
_________________________________________________________________
resnet152 (Functional)       (None, 8, 1, 2048)        58370944  
_________________________________________________________________
flatten (Flatten)            (None, 16384)             0         
_________________________________________________________________
regression_head_1 (Dense)    (None, 256)               4194560   
Total params: 62,565,513
Trainable params: 62,414,089
Non-trainable params: 151,424
____________________________________